# SVM Implementation

In [6]:
import os
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.svm import LinearSVC, SVC
import joblib
from itertools import product

# Load dataset
df = pd.read_csv("loan_approval_dataset.csv")
df.drop(columns=["Id"], inplace=True)
df["Risk_Flag"] = df["Risk_Flag"].apply(lambda x: 1 if x == 1 else -1)

X = df.drop(columns=["Risk_Flag"])
y = df["Risk_Flag"]

# One-hot encode categorical features
X = pd.get_dummies(X)

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=9
)

In [3]:
linear_params = {
    "C": [0.1, 1.0, 10.0]
}

# For polynomial kernel SVM (NonLinear_SVM)
poly_params = {
    "C": [0.1, 1.0, 10.0],
    "degree": [2, 3],
    "gamma": [0.001, 0.01]
}

# For RBF kernel SVM (Kernel_SVM)
rbf_params = {
    "C": [0.1, 1.0, 10.0],
    "gamma": [0.001, 0.01]
}

weights_dir = "SVM_Weights"
os.makedirs(weights_dir, exist_ok=True)
model_info_csv_path = "SVM_Model_Info.csv"

# If the CSV exists, load it; otherwise, start with an empty DataFrame.
if os.path.exists(model_info_csv_path):
    combined_df = pd.read_csv(model_info_csv_path)
else:
    combined_df = pd.DataFrame()

In [4]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, pos_label=1, zero_division=0)
    precision = precision_score(y_test, y_pred, pos_label=1, zero_division=0)
    f1 = f1_score(y_test, y_pred, pos_label=1, zero_division=0)
    
    # Compute hinge loss if decision_function is available.
    if hasattr(model, "decision_function"):
        decision_values = model.decision_function(X_test)
    else:
        decision_values = model.decision_function(X_test)
    y_test_np = np.where(np.array(y_test)==1, 1, -1)
    hinge_losses = np.maximum(0, 1 - y_test_np * decision_values)
    avg_loss = np.mean(hinge_losses)
    return accuracy, recall, precision, f1, avg_loss

In [5]:
model_info_list = []

for C in linear_params["C"]:
    model_name = f"Linear_SVM_C{C}"
    print(f"\nTraining {model_name}...")
    
    model = LinearSVC(C=C, random_state=9, max_iter=10000, verbose=1)
    model.fit(X_train, y_train)
    
    # Save model weights
    save_path = os.path.join(weights_dir, f"{model_name}_model.pkl")
    joblib.dump(model, save_path)
    print(f"Finished training {model_name}. Model saved to {save_path}")
    
    # Evaluate
    accuracy, recall, precision, f1, avg_loss = evaluate_model(model, X_test, y_test)
    print(f"Evaluation for {model_name} - Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1-score: {f1:.4f}, Hinge Loss: {avg_loss:.4f}")
    
    model_info_list.append({
        "Model": model_name,
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1_score": f1,
        "loss": avg_loss
    })


Training Linear_SVM_C0.1...
[LibLinear]Finished training Linear_SVM_C0.1. Model saved to SVM_Weights\Linear_SVM_C0.1_model.pkl
Evaluation for Linear_SVM_C0.1 - Accuracy: 0.8804, Recall: 0.0000, Precision: 0.0000, F1-score: 0.0000, Hinge Loss: 0.4174

Training Linear_SVM_C1.0...
[LibLinear]Finished training Linear_SVM_C1.0. Model saved to SVM_Weights\Linear_SVM_C1.0_model.pkl
Evaluation for Linear_SVM_C1.0 - Accuracy: 0.8804, Recall: 0.0000, Precision: 0.0000, F1-score: 0.0000, Hinge Loss: 0.4174

Training Linear_SVM_C10.0...
[LibLinear]Finished training Linear_SVM_C10.0. Model saved to SVM_Weights\Linear_SVM_C10.0_model.pkl
Evaluation for Linear_SVM_C10.0 - Accuracy: 0.8804, Recall: 0.0000, Precision: 0.0000, F1-score: 0.0000, Hinge Loss: 0.4174


In [ ]:
for C, degree, gamma in product(poly_params["C"], poly_params["degree"], poly_params["gamma"]):
    model_name = f"NonLinear_SVM_poly_C{C}_deg{degree}_gamma{gamma}"
    print(f"\nTraining {model_name}...")
    
    model = SVC(kernel='poly', C=C, degree=degree, gamma=gamma, random_state=9, verbose=True)
    model.fit(X_train, y_train)
    
    save_path = os.path.join(weights_dir, f"{model_name}_model.pkl")
    joblib.dump(model, save_path)
    print(f"Finished training {model_name}. Model saved to {save_path}")
    
    accuracy, recall, precision, f1, avg_loss = evaluate_model(model, X_test, y_test)
    print(f"Evaluation for {model_name} - Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1-score: {f1:.4f}, Hinge Loss: {avg_loss:.4f}")
    
    model_info_list.append({
        "Model": model_name,
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1_score": f1,
        "loss": avg_loss
    })


Training NonLinear_SVM_poly_C0.1_deg2_gamma0.001...
[LibSVM]

In [ ]:
for C, gamma in product(rbf_params["C"], rbf_params["gamma"]):
    model_name = f"Kernel_SVM_rbf_C{C}_gamma{gamma}"
    print(f"\nTraining {model_name}...")
    
    model = SVC(kernel='rbf', C=C, gamma=gamma, random_state=9, verbose=True)
    model.fit(X_train, y_train)
    
    save_path = os.path.join(weights_dir, f"{model_name}_model.pkl")
    joblib.dump(model, save_path)
    print(f"Finished training {model_name}. Model saved to {save_path}")
    
    accuracy, recall, precision, f1, avg_loss = evaluate_model(model, X_test, y_test)
    print(f"Evaluation for {model_name} - Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1-score: {f1:.4f}, Hinge Loss: {avg_loss:.4f}")
    
    model_info_list.append({
        "Model": model_name,
        "accuracy": accuracy,
        "recall": recall,
        "precision": precision,
        "f1_score": f1,
        "loss": avg_loss
    })

new_df = pd.DataFrame(model_info_list)
combined_df = pd.concat([combined_df, new_df], ignore_index=True)
combined_df.to_csv(model_info_csv_path, index=False)
print(f"\nModel evaluation info appended to {model_info_csv_path}")

### This was the original training loop for hyperparameter tuning. Ignore for now. I was using PyTorch but transitioned to SKLearn for simplicity's sake. 

In [ ]:
# Define hyperparameter grids
optimizer_dict = {
    "SGD": lambda params, lr: optim.SGD(params, lr=lr),
    "Adam": lambda params, lr: optim.Adam(params, lr=lr),
    "RMSprop": lambda params, lr: optim.RMSprop(params, lr=lr)
}

learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [4, 8, 16, 32, 64]
epoch_options = [5, 10, 20]

results = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loop over each combination of hyperparameters using itertools.product
for opt_name, lr, batch_size, num_epochs in itertools.product(optimizer_dict.keys(),
                                                               learning_rates,
                                                               batch_sizes,
                                                               epoch_options):
    # Create DataLoaders for current batch size
    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Reinitialize the model for each run and move to device
    model = SVM(X_train.shape[1]).to(device)
    optimizer = optimizer_dict[opt_name](model.parameters(), lr)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        for batch_x, batch_y in train_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = hinge_loss(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(train_loader)
        # Optionally, print progress for each run
        print(f"[{opt_name}, lr={lr}, batch_size={batch_size}, epochs={num_epochs}] "
              f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")
    
    # Evaluation on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            outputs = model(batch_x)
            predicted = torch.where(outputs >= 0, torch.tensor(1.0, device=device), 
                                    torch.tensor(-1.0, device=device))
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
    accuracy = 100 * correct / total
    
    # Save the results for this configuration
    results.append({
        "optimizer": opt_name,
        "lr": lr,
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "final_loss": avg_loss,
        "test_accuracy": accuracy
    })

# Convert the results to a DataFrame and save as CSV
results_df = pd.DataFrame(results)
results_df.to_csv("grid_search_results.csv", index=False)
print("Results saved to grid_search_results.csv")

[SGD, lr=0.001, batch_size=4, epochs=5] Epoch [1/5], Loss: 0.2742
[SGD, lr=0.001, batch_size=4, epochs=5] Epoch [2/5], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=5] Epoch [3/5], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=5] Epoch [4/5], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=5] Epoch [5/5], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [1/10], Loss: 0.2753
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [2/10], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [3/10], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [4/10], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [5/10], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [6/10], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [7/10], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [8/10], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [9/10], Loss: 0.2604
[SGD, lr=0.001, batch_size=4, epochs=10] Epoch [10/10], Lo